In [5]:
from Bio import SearchIO, Entrez, SeqIO
from ete3 import NCBITaxa, Tree
# upload database when necessary
#ncbi.update_taxonomy_database()

ncbi = NCBITaxa()

In [2]:
domtblout_file = "data/OG_343.domtblout"

### Parse hmmsearch results and taxonomy
Parse the file. Get the taxonomy from the last part of the description, ie. `[Siphoviridae sp.]`. Filter out hits that are not Bacteria or Virus.

In [9]:
target_proteins = list()
target_taxids   = list()
target_proteins_taxids = dict()
target_proteins_lineages = list()

records = SearchIO.parse(domtblout_file, "hmmsearch3-domtab")
for record in records:
    og_len = record.seq_len
    # iterate the protein hits
    for hit in record.hits:
        hit_len = hit.seq_len
        if hit.evalue < 0.01:
            # set up the lists for the coverage calculation
            hit_starts = list()
            hit_ends   = list()
            og_starts = list()
            og_ends   = list()
            for hsp in hit.hsps:
                if hsp.evalue < 0.01:
                    hit_starts.append(hsp.env_start) 
                    hit_ends.append(hsp.env_end)
                    og_starts.append(hsp.query_start)
                    og_ends.append(hsp.query_end)
            #print(hit.id, hit_starts, hit_ends, og_starts, og_ends)
                
            # check there were hits passing the i-value thrshold
            if hit_starts:
                # calculate coverage of hit (protein seq) and query (og)
                intervals_hit = [[s,e] for s,e in zip(hit_starts, hit_ends)]
                intervals_og = [[s,e] for s,e in zip(og_starts, og_ends)]
               
                intervals_hit.sort(key=lambda interval: interval[0])
                intervals_og.sort(key=lambda interval: interval[0])

                merged_hit = [intervals_hit[0]]
                for current in intervals_hit:
                    previous = merged_hit[-1]
                    if current[0] <= previous[1]:
                        previous[1] = max(previous[1], current[1])
                    else:
                        merged_hit.append(current)

                covered_length_hit = float(0)
                for interval in merged_hit:
                    covered_length_hit += (interval[1] - interval[0])

                coverage_hit = float(covered_length_hit/hit_len)


                merged_og = [intervals_og[0]]
                for current in intervals_og:
                    previous = merged_og[-1]
                    if current[0] <= previous[1]:
                        previous[1] = max(previous[1], current[1])
                    else:
                        merged_og.append(current)

                covered_length_og = float(0)
                for interval in merged_og:
                    covered_length_og += (interval[1] - interval[0])

                coverage_og = float(covered_length_og/og_len)

                    
                # check that coverages pass the cutoff
                if coverage_og >= 0.5 and coverage_hit >= 0.5:
                    # check which is the taxonomy
                    taxa_description = hit.description.split(" [")[-1][:-1]
                    # line below returns a dict, k=taxa_description  v=[tax_id]
                    tax_id_dict = ncbi.get_name_translator([taxa_description])
                    tax_id = tax_id_dict.get(taxa_description)[0]
                    # get the lineage of the tax id
                    lineage = ncbi.get_lineage(tax_id)
                    # check if it is bacteria or virus
                    if 2 in lineage or 10239 in lineage:
                        # EXCLUDE HITS FROM CRASS-LIKE PHAGES
                        if 1978007 not in lineage:
                            target_proteins.append(hit.id)
                            target_taxids.append(tax_id)
                            target_proteins_taxids[hit.id] = tax_id
                            target_proteins_lineages += lineage

print(target_proteins)

['DAW89363.1', 'DAW86324.1', 'DAW94066.1', 'DAU94425.1', 'DAF01075.1', 'DAR20966.1', 'DAW20005.1', 'DAP51448.1', 'DAR70384.1', 'DAX21802.1', 'DAR45680.1', 'DAQ45717.1', 'DAW07241.1', 'DAL80299.1', 'DAE28469.1', 'DAX32588.1', 'DAP34063.1', 'DAJ75044.1', 'DAH06617.1', 'DAP92696.1', 'DAJ18143.1', 'DAO43059.1', 'DAH51560.1', 'DAX07875.1', 'WP_153163798.1', 'MBS6796132.1', 'DAV68353.1', 'WP_153170287.1', 'DAL58409.1', 'DAX15220.1', 'DAO36476.1', 'MBR5795345.1', 'MBQ7819940.1', 'MBS7122782.1', 'DAL29697.1', 'DAQ34250.1', 'CCX55664.1', 'DAJ12619.1', 'DAF73599.1', 'NBK99050.1', 'DAH77719.1', 'DAS74269.1', 'DAL04585.1', 'DAS78946.1', 'NBK97982.1']


### Download proteins from NCBI
First download the GenBank records, then parse the files to fasta.

In [2]:
# Download hits in GenBank format
target_proteins = ['DAW89363.1', 'DAW86324.1', 'DAW94066.1', 'DAU94425.1', 'DAF01075.1', 'DAR20966.1', 'DAW20005.1', 'DAP51448.1', 'DAR70384.1', 'DAX21802.1', 'DAR45680.1', 'DAQ45717.1', 'DAW07241.1', 'DAL80299.1', 'DAE28469.1', 'DAX32588.1', 'DAP34063.1', 'DAJ75044.1', 'DAH06617.1', 'DAP92696.1', 'DAJ18143.1', 'DAO43059.1', 'DAH51560.1', 'DAX07875.1', 'WP_153163798.1', 'MBS6796132.1', 'DAV68353.1', 'WP_153170287.1', 'DAL58409.1', 'DAX15220.1', 'DAO36476.1', 'MBR5795345.1', 'MBQ7819940.1', 'MBS7122782.1', 'DAL29697.1', 'DAQ34250.1', 'CCX55664.1', 'DAJ12619.1', 'DAF73599.1', 'NBK99050.1', 'DAH77719.1', 'DAS74269.1', 'DAL04585.1', 'DAS78946.1', 'NBK97982.1']
Entrez.email   = 'dcarrillo.bioinf@gmail.com'
Entrez.api_key = 'b6db7fece605d37fcabd4b93749d2e46aa09'

with open("borrar.gb", "w") as fout:
    handle = Entrez.efetch(db="protein", id=target_proteins, rettype="gb", retmode="text")
    batch_data = handle.read()
    fout.write(batch_data)
    handle.close()

In [19]:
# parse the GenBank file and get FASTA sequences

to_faa = list()

records = SeqIO.parse("data/OG_343_NCBI.gb", "gb")

for record in records:
    # put the id in this form: <TAXID>|<SEQID>
    record.id = f"{target_proteins_taxids[record.id]}|{record.id}"
    record.description = ""
    to_faa.append(record)
    
with open("data/OG_343_NCBI.faa", "w") as fout:
    SeqIO.write(to_faa, fout, "fasta")

### Merge OG and NCBI, align, trim and fasttree

```
(mafft_env) danielc@encode:~/projects/Bas_phages/crassphage_proteins/notung_pipeline/data:cat ../../../3_Broccoli/4_easy_OGs/0_easy_OGs_faa/OG_343.faa OG_343_NCBI.faa > gene_tree.faa

(mafft_env) danielc@encode:~/projects/Bas_phages/crassphage_proteins/notung_pipeline/data:mafft-einsi --thread 10 gene_tree.faa > gene_tree.mafft

(mafft_env) danielc@encode:~/projects/Bas_phages/crassphage_proteins/notung_pipeline/data:trimal -in gene_tree.mafft -out gene_tree_trim.mafft -gt 0.9

(phylogenies) danielc@encode:~/projects/Bas_phages/crassphage_proteins/notung_pipeline/data:fasttree gene_tree_trim.mafft > gene_tree_trim.nwk
```

Now parse the tree and change leaves names to only the genome_id for the reference crassphages, or the taxid for the NCBI hits:

In [95]:
# read TerL tree
gene_tree = Tree("data/gene_tree_trim.nwk", format=1)
# changes leaves' names
for leaf in gene_tree.iter_leaves():
    # check if the leaf comes from the reference set
    label = leaf.name.split("|")[0]
    leaf.name = label
# write the final tree
gene_tree.write(format=1, outfile="data/gene_tree_final.nwk")

### Create species tree
I have stored in a list all the tax_ids for the proteins that passed the cutoffs. Create a tree with them and attach it to the crAssphage TerL tree.

In [6]:
# make tax_ids unique
target_taxids = list(set(target_taxids))

# add crAss-like taxid 
target_taxids.append(1978007)

# get the ncbi tree for these tax_ids
ncbi_topology = ncbi.get_topology(target_taxids)
print(ncbi_topology.get_ascii(attributes=["sci_name"]))
# root the tree manually
ncbi_topology_nwk = ncbi_topology.write()
ncbi_topology_nwk = ncbi_topology_nwk[:-1] + "root;"
ncbi_tree = Tree(ncbi_topology_nwk, format=1)
print(ncbi_tree)
ncbi_tree.show()


                                         /-Bacteriophage sp.
           /unclassified bacterial viruses
          |                              \-virus sp. ct9pU4
          |
          |                                    /-Podoviridae sp. ctY3D12
    /Viruses                                  |
   |      |            /unclassified Podoviridae-Podoviridae sp.
   |      |           |                       |
   |      |           |                        \-crAss-like viruses
   |       \Caudovirales
   |                  |--Myoviridae sp.
   |                  |
   |                   \-Siphoviridae sp.
-root
   |                                                    /-Erysipelotrichaceae bacterium
   |                                  /Erysipelotrichales
   |                  /Erysipelotrichia                 \-Coprobacillus sp.
   |                 |               |
   |        /Firmicutes               \-Erysipelotrichia bacterium
   |       |         |
   |       |          \-Veillonel

In [88]:
# read taxonomy file and save it in a dict
crass_taxonomy = dict()

tax_file = "data/crass_taxonomy.txt"
lines = [line.strip().split("\t") for line in open(tax_file).readlines()]
for line in lines:
    crass_taxonomy[line[0]] = line[2]

families = list(set(crass_taxonomy.values()))
families.remove("outgroup")

# read TerL tree
terl_tree = Tree("data/species_tree.nwk", format=1)
# assign taxonomy
for leaf in terl_tree.iter_leaves():
    # check if the leaf comes from the reference set
    genome = leaf.name.split("|")[0]   
    leaf.add_features(family=crass_taxonomy[genome], genome=genome)



outgs_leaves = terl_tree.search_nodes(family="outgroup")
outgs_lca = terl_tree.get_common_ancestor(outgs_leaves)
# reroot the tree 
terl_tree.set_outgroup(outgs_lca)

# change leaves names to genomes_ids only
for leaf in terl_tree.iter_leaves():
    leaf.name = leaf.genome
    
## collapse nodes, just for ease of visualization
#for family in families:
#    fam_leaves = terl_tree.search_nodes(family=family)
#    fam_lca = terl_tree.get_common_ancestor(fam_leaves)
#    fam_lca.name = family
#    for child in fam_lca.get_children():
#        child.detach()



In [90]:
#print(terl_tree)

In [91]:
# merge both trees in the crAss-like clade of the ncbi tree
ncbi_crass_node = ncbi_tree.search_nodes(name="1978007")[0]
ncbi_crass_node.add_child(terl_tree)

Tree node '' (0x7f79f53f5d3)

In [93]:
#print(ncbi_tree)

In [94]:
# write to final species tree file
ncbi_tree.write(format=1, outfile="data/species_tree_final.nwk")

## TESTS

In [15]:
a = list(set(target_proteins_lineages))

In [4]:

target_taxids = [57723, 976, 1239, 1224]

# add crAss-like taxid 
target_taxids.append(1978007)

# get the ncbi tree for these tax_ids
ncbi_topology = ncbi.get_topology(target_taxids)
print(ncbi_topology.get_ascii(attributes=["sci_name"]))
# root the tree manually
ncbi_topology_nwk = ncbi_topology.write()
ncbi_topology_nwk = ncbi_topology_nwk[:-1] + "root;"
ncbi_tree = Tree(ncbi_topology_nwk, format=1)
#print(ncbi_tree)
#ncbi_tree.show()


            /-Proteobacteria
           |
           |--Acidobacteria
    /Bacteria
   |       |--Firmicutes
-root      |
   |        \-Bacteroidetes
   |
    \-crAss-like viruses
